In [6]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
import pickle as pkl
from sklearn.linear_model import LogisticRegression

In [7]:
#function to fromat columns, this was initially going to be 
#for changeing the target variable to class but 
#I started doing the adhoc
def changetorisk(df,col):
    df[col] = df[col].apply(lambda x: x[2])
    df[col] = df[col].astype(int)
#     df[col] = df[col].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')
    return df
Column_names = ['Age','Gender','Education'
                ,'Country','Ethnicity'
                ,'Nscore','Escore','Oscore'
                ,'Ascore','Cscore','Impulsive'
                ,'SensationSeeking','Alchol'
                ,'Amphet','Amyl','Benzos'
                ,'Caff','Cannibis','Chocalate'
                ,'Coke','Crack','Ecstasy','Heroin'
                ,'Ketamine','Legalh','Lsd','Meth'
                ,'Mushroom','Nicotine','Semer','VSA']

In [8]:
df = pd.read_csv('drug_consumption.csv', names = Column_names)
#load in source data

In [9]:
#make a list of the columns for which I can make a potential target var
drugcols = ['Alchol', 'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannibis', 'Chocalate', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'Lsd', 'Meth', 'Mushroom', 'Nicotine', 'Semer', 'VSA']
print(drugcols)

['Alchol', 'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannibis', 'Chocalate', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'Lsd', 'Meth', 'Mushroom', 'Nicotine', 'Semer', 'VSA']


In [10]:
#using the change ris to turn the classes into their porpotionate 
#so that I can do math to make a potential target
for i in drugcols:
    df = changetorisk(df,i)
df

,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,...,Ecstasy,Heroin,Ketamine,Legalh,Lsd,Meth,Mushroom,Nicotine,Semer,VSA
1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,0,0,0,0,0,0,0,2,0,0
2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,4,0,2,0,2,3,0,4,0,0
3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,0,0,0,0,0,0,1,0,0,0
4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,0,0,2,0,0,0,0,2,0,0
5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,1,0,0,1,0,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,...,0,0,0,3,3,0,0,0,0,5
1885,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,...,2,0,0,3,5,4,4,5,0,0
1886,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,...,4,0,2,0,2,0,2,6,0,0
1887,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,...,3,0,0,3,3,0,3,4,0,0


In [11]:
df['Gender'] = df['Gender'].apply(lambda x: 'Female' if x >0 else 'Male')
df = df.drop(['Country','Ethnicity'], axis = 1)
#didnt want to use these parameters at first 

In [12]:
df

,Age,Gender,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,...,Ecstasy,Heroin,Ketamine,Legalh,Lsd,Meth,Mushroom,Nicotine,Semer,VSA
1,0.49788,Female,-0.05921,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,...,0,0,0,0,0,0,0,2,0,0
2,-0.07854,Male,1.98437,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,...,4,0,2,0,2,3,0,4,0,0
3,0.49788,Male,-0.05921,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,...,0,0,0,0,0,0,1,0,0,0
4,-0.95197,Female,1.16365,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,...,0,0,2,0,0,0,0,2,0,0
5,0.49788,Female,1.98437,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,...,1,0,0,1,0,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,-0.95197,Female,-0.61113,-1.19430,1.74091,1.88511,0.76096,-1.13788,0.88113,1.92173,...,0,0,0,3,3,0,0,0,0,5
1885,-0.95197,Male,-0.61113,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.88113,0.76540,...,2,0,0,3,5,4,4,5,0,0
1886,-0.07854,Female,0.45468,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,0.52975,-0.52593,...,4,0,2,0,2,0,2,6,0,0
1887,-0.95197,Female,-0.61113,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.29221,1.22470,...,3,0,0,3,3,0,3,4,0,0


In [13]:
#next few cells are dealing with the fact that the data has been standardized
#this is fine for modeling cause I will do this anyway but in eda I want
#my data to be alittle more meaningfull 
df.Age = df.Age.astype(float)
df.Age.value_counts()


-0.95197    643
-0.07854    481
 0.49788    356
 1.09449    294
 1.82213     93
 2.59171     18
Name: Age, dtype: int64

In [14]:
agecases = [df.Age < -0.85197, df.Age == -0.07854,df.Age < 0.89788
            ,df.Age == 1.09449,df.Age == 1.82213,df.Age == 2.59171]
agechoices = ["18-24","25-34","35-44","45-54","55-64","65+"]

In [15]:
df.Age = np.select(agecases,agechoices)

In [16]:
df.Age.value_counts()

18-24    643
25-34    481
35-44    356
45-54    294
55-64     93
65+       18
Name: Age, dtype: int64

In [17]:
-2.43591 'Left school before 16 years' 
-1.73790 'Left school at 16 years' 
-1.43719 'Left school at 17 years'
-1.22751 'Left school at 18 years '
-0.61113 'Some college or university, no certificate or degree '
-0.05921' Professional certificate/ diploma '
0.45468 'University degree '
1.16365 'Masters degree '
1.98437 'Doctorate degree '

SyntaxError: invalid syntax (<ipython-input-17-a7570468d2a5>, line 1)

In [18]:
educases = [df.Education < -1.85197, df.Education == -1.73790,df.Education < -1.33719
            ,df.Education < -1.12751,df.Education < -0.51113,df.Education < -0.04921
            ,df.Education < 0.55468,df.Education < 1.2365,df.Education < 2.98437]
educhoices = ['Left school before 16 years'
              ,'Left school at 16 years' 
              ,'Left school at 17 years'
              ,'Left school at 18 years '
              ,'Some college or university, no certificate or degree'
              ,'Professional certificate/diploma'
              ,'University degree'
              ,'Masters degree'
              ,'Doctorate degree']

In [19]:
df.Education =np.select(educases,educhoices)

In [20]:
df.Education.value_counts()

Some college or university, no certificate or degree    506
University degree                                       480
Masters degree                                          283
Professional certificate/diploma                        270
Left school at 18 years                                 100
Left school at 16 years                                  99
Doctorate degree                                         89
Left school at 17 years                                  30
Left school before 16 years                              28
Name: Education, dtype: int64

In [21]:
sorted(list(df.Nscore.unique()))

[-3.46436,
 -3.15735,
 -2.75696,
 -2.52197,
 -2.42317,
 -2.3436,
 -2.21844,
 -2.05048,
 -1.86962,
 -1.69163,
 -1.55078,
 -1.43907,
 -1.32828,
 -1.1943,
 -1.05308,
 -0.92104,
 -0.79151,
 -0.67825,
 -0.58016,
 -0.46725,
 -0.34799,
 -0.24649,
 -0.14882,
 -0.05188,
 0.04257,
 0.13606,
 0.22393,
 0.31287,
 0.41667,
 0.52135,
 0.62967,
 0.73545,
 0.82562,
 0.91093,
 1.02119,
 1.13281,
 1.23461,
 1.37297,
 1.49158,
 1.60383,
 1.72012,
 1.8399,
 1.98437,
 2.127,
 2.28554,
 2.46262,
 2.61139,
 2.82196,
 3.27393]

In [22]:
list(range(12,61))

[12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]

In [23]:
df.Nscore.replace(to_replace=sorted(list(df.Nscore.unique())),value=list(range(12,61))
                  ,inplace = True)
df.Nscore.astype(int)

1       39
2       29
3       31
4       34
5       43
        ..
1884    25
1885    33
1886    47
1887    45
1888    31
Name: Nscore, Length: 1885, dtype: int32

In [24]:
Escorenumlist = [16, 18, 19
           , 20, 21, 22, 23, 24, 25, 26, 27
           , 28, 29, 30, 31, 32, 33, 34, 35
           , 36, 37, 38, 39, 40, 41, 42, 43
           , 44, 45, 46, 47, 48, 49, 50, 51
           , 52, 53, 54, 55, 56, 58, 59]


In [25]:
df.Escore.replace(to_replace=sorted(list(df.Escore.unique())),value=Escorenumlist
                  ,inplace = True)
df.Escore.astype(int)

1       36
2       52
3       45
4       34
5       28
        ..
1884    51
1885    51
1886    30
1887    26
1888    53
Name: Escore, Length: 1885, dtype: int32

In [26]:
Oscorenumlist = [ 24, 26,
           28, 29, 30, 31, 32, 33, 34, 35
           , 36, 37, 38, 39, 40, 41, 42, 43
           , 44, 45, 46, 47, 48, 49, 50, 51
           , 52, 53, 54, 55, 56,57, 58, 59,60]


In [27]:
df.Oscore.replace(to_replace=sorted(list(df.Oscore.unique())),value=Oscorenumlist
                  ,inplace = True)
df.Oscore.astype(int)

1       42
2       55
3       40
4       46
5       43
        ..
1884    57
1885    50
1886    37
1887    48
1888    56
Name: Oscore, Length: 1885, dtype: int32

In [28]:
Ascorenumlist = [ 12,16,18,23, 24,25, 26,27,
           28, 29, 30, 31, 32, 33, 34, 35
           , 36, 37, 38, 39, 40, 41, 42, 43
           , 44, 45, 46, 47, 48, 49, 50, 51
           , 52, 53, 54, 55, 56,57, 58, 59,60]

In [29]:
df.Ascore.replace(to_replace=sorted(list(df.Ascore.unique())),value=Ascorenumlist
                  ,inplace = True)
df.Ascore.astype(int)

1       37
2       48
3       32
4       47
5       41
        ..
1884    48
1885    48
1886    31
1887    32
1888    50
Name: Ascore, Length: 1885, dtype: int32

In [30]:
Cscorenumlist = [17,19,20,21,22,23, 24,25, 26,27,
           28, 29, 30, 31, 32, 33, 34, 35
           , 36, 37, 38, 39, 40, 41, 42, 43
           , 44, 45, 46, 47, 48, 49, 50, 51
           , 52, 53, 54, 55, 56,57, 59]

In [31]:
df.Cscore.replace(to_replace=sorted(list(df.Cscore.unique())),value=Cscorenumlist
                  ,inplace = True)
df.Cscore.astype(int)

1       42
2       41
3       34
4       46
5       50
        ..
1884    33
1885    30
1886    31
1887    22
1888    45
Name: Cscore, Length: 1885, dtype: int32

In [32]:
df.Impulsive.replace(to_replace = sorted(list(df.Impulsive.unique())),value = list(range(1,11))
                    ,inplace = True)


In [33]:
drugcols=[ 'Amphet', 'Amyl', 'Benzos', 'Cannibis',  'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'Lsd', 'Meth', 'Mushroom', 'Semer', 'VSA']

In [34]:
df.SensationSeeking.replace(to_replace = sorted(list(df.SensationSeeking.unique())),value = list(range(1,12))
                    ,inplace = True)

In [35]:
#split up potential dataframes by target for ease of use for me later
stimulant_columns = ['Amphet','Coke','Crack','Ketamine']
stimdf = df.drop(drugcols,axis = 1)
stimdf['stim'] = (df.Amphet  + df.Coke + df.Crack + df.Ketamine)/4
#stimdf['stim'] = stimdf['stim'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')

In [36]:
depressant_columns = ['Amyl','Benzos','Cannibis','Ketamine']
depressantdf = df.drop(drugcols,axis = 1)
depressantdf['depress'] = (df.Amyl  + df.Benzos )/2
#depressantdf['depress'] = depressantdf['depress'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')

In [37]:
hallucinagen_columns = ['Ecstacy','Lsd','Mushroom']
hallucinagendf =df.drop(drugcols,axis = 1)
hallucinagendf['hallu'] = (df.Ecstasy  + df.Lsd + df.Mushroom )/3
#hallucinagendf['hallu'] = hallucinagendf['hallu'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')


In [38]:
other_columns = ['Amyl','Benzos','Cannibis','Ketamine']
df['depress'] = (df.Amyl  + df.Benzos )/2
df['depress'] = df['depress'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 2 else 'high')

In [39]:
df['stimulant'] = stimdf['stim']
df['depressant'] = depressantdf['depress']
df['hallucinagen'] = hallucinagendf['hallu']

In [40]:
df

,Age,Gender,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,...,Lsd,Meth,Mushroom,Nicotine,Semer,VSA,depress,stimulant,depressant,hallucinagen
1,35-44,Female,Professional certificate/diploma,39.0,36.0,42.0,37.0,42.0,4.0,3.0,...,0,0,0,2,0,0,low,0.50,1.0,0.000000
2,25-34,Male,Doctorate degree,29.0,52.0,55.0,48.0,41.0,3.0,6.0,...,2,3,0,4,0,0,low,1.75,1.0,2.000000
3,35-44,Male,Professional certificate/diploma,31.0,45.0,40.0,32.0,34.0,2.0,8.0,...,0,0,1,0,0,0,None,0.00,0.0,0.333333
4,18-24,Female,Masters degree,34.0,34.0,46.0,47.0,46.0,2.0,3.0,...,0,0,0,2,0,0,low,1.00,1.5,0.000000
5,35-44,Female,Doctorate degree,43.0,28.0,43.0,41.0,50.0,4.0,6.0,...,0,0,2,2,0,0,low,0.25,0.5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,18-24,Female,"Some college or university, no certificate or ...",25.0,51.0,57.0,48.0,33.0,7.0,11.0,...,3,0,0,0,0,5,None,0.00,0.0,1.000000
1885,18-24,Male,"Some college or university, no certificate or ...",33.0,51.0,50.0,48.0,30.0,7.0,9.0,...,5,4,4,5,0,0,None,0.00,0.0,3.666667
1886,25-34,Female,University degree,47.0,30.0,37.0,31.0,31.0,6.0,5.0,...,2,0,2,6,0,0,high,3.00,5.0,2.666667
1887,18-24,Female,"Some college or university, no certificate or ...",45.0,26.0,48.0,32.0,22.0,8.0,10.0,...,3,0,3,4,0,0,None,0.00,0.0,3.000000


In [41]:
#decided that I wanted to predict on stimulant so I made another df 
#for stim to be ready, I know this is a mess..

In [42]:
stimdf.to_csv('StimDf.csv')
depressantdf.to_csv('depressantdf.csv')
hallucinagendf.to_csv('hallucinagendf.csv')
cleardf = df.drop(drugcols,axis = 1)
cleardf = cleardf.drop('depress',axis = 1)
cleardf.to_csv('cleardf.csv')

In [43]:
cleardf

,Age,Gender,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Caff,Chocalate,Nicotine,stimulant,depressant,hallucinagen
1,35-44,Female,Professional certificate/diploma,39.0,36.0,42.0,37.0,42.0,4.0,3.0,5,6,5,2,0.50,1.0,0.000000
2,25-34,Male,Doctorate degree,29.0,52.0,55.0,48.0,41.0,3.0,6.0,5,6,6,4,1.75,1.0,2.000000
3,35-44,Male,Professional certificate/diploma,31.0,45.0,40.0,32.0,34.0,2.0,8.0,6,6,4,0,0.00,0.0,0.333333
4,18-24,Female,Masters degree,34.0,34.0,46.0,47.0,46.0,2.0,3.0,4,5,4,2,1.00,1.5,0.000000
5,35-44,Female,Doctorate degree,43.0,28.0,43.0,41.0,50.0,4.0,6.0,4,6,6,2,0.25,0.5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,18-24,Female,"Some college or university, no certificate or ...",25.0,51.0,57.0,48.0,33.0,7.0,11.0,5,4,4,0,0.00,0.0,1.000000
1885,18-24,Male,"Some college or university, no certificate or ...",33.0,51.0,50.0,48.0,30.0,7.0,9.0,5,5,4,5,0.00,0.0,3.666667
1886,25-34,Female,University degree,47.0,30.0,37.0,31.0,31.0,6.0,5.0,4,6,6,6,3.00,5.0,2.666667
1887,18-24,Female,"Some college or university, no certificate or ...",45.0,26.0,48.0,32.0,22.0,8.0,10.0,5,6,5,4,0.00,0.0,3.000000


In [44]:

datadf = cleardf
datadf

,Age,Gender,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Caff,Chocalate,Nicotine,stimulant,depressant,hallucinagen
1,35-44,Female,Professional certificate/diploma,39.0,36.0,42.0,37.0,42.0,4.0,3.0,5,6,5,2,0.50,1.0,0.000000
2,25-34,Male,Doctorate degree,29.0,52.0,55.0,48.0,41.0,3.0,6.0,5,6,6,4,1.75,1.0,2.000000
3,35-44,Male,Professional certificate/diploma,31.0,45.0,40.0,32.0,34.0,2.0,8.0,6,6,4,0,0.00,0.0,0.333333
4,18-24,Female,Masters degree,34.0,34.0,46.0,47.0,46.0,2.0,3.0,4,5,4,2,1.00,1.5,0.000000
5,35-44,Female,Doctorate degree,43.0,28.0,43.0,41.0,50.0,4.0,6.0,4,6,6,2,0.25,0.5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,18-24,Female,"Some college or university, no certificate or ...",25.0,51.0,57.0,48.0,33.0,7.0,11.0,5,4,4,0,0.00,0.0,1.000000
1885,18-24,Male,"Some college or university, no certificate or ...",33.0,51.0,50.0,48.0,30.0,7.0,9.0,5,5,4,5,0.00,0.0,3.666667
1886,25-34,Female,University degree,47.0,30.0,37.0,31.0,31.0,6.0,5.0,4,6,6,6,3.00,5.0,2.666667
1887,18-24,Female,"Some college or university, no certificate or ...",45.0,26.0,48.0,32.0,22.0,8.0,10.0,5,6,5,4,0.00,0.0,3.000000


In [45]:
datadf.Age = datadf.Age.replace(agechoices,list(range(1,7)))
datadf.Education = datadf.Education.replace(educhoices,list(range(-4,5,1)))

In [46]:
datadf = pd.get_dummies(datadf,columns=['Gender'],drop_first=True)

In [47]:
datadf = datadf.drop('Gender_Male',axis=1)

In [48]:
datadf

,Age,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Caff,Chocalate,Nicotine,stimulant,depressant,hallucinagen
1,3,1,39.0,36.0,42.0,37.0,42.0,4.0,3.0,5,6,5,2,0.50,1.0,0.000000
2,2,4,29.0,52.0,55.0,48.0,41.0,3.0,6.0,5,6,6,4,1.75,1.0,2.000000
3,3,1,31.0,45.0,40.0,32.0,34.0,2.0,8.0,6,6,4,0,0.00,0.0,0.333333
4,1,3,34.0,34.0,46.0,47.0,46.0,2.0,3.0,4,5,4,2,1.00,1.5,0.000000
5,3,4,43.0,28.0,43.0,41.0,50.0,4.0,6.0,4,6,6,2,0.25,0.5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,1,0,25.0,51.0,57.0,48.0,33.0,7.0,11.0,5,4,4,0,0.00,0.0,1.000000
1885,1,0,33.0,51.0,50.0,48.0,30.0,7.0,9.0,5,5,4,5,0.00,0.0,3.666667
1886,2,2,47.0,30.0,37.0,31.0,31.0,6.0,5.0,4,6,6,6,3.00,5.0,2.666667
1887,1,0,45.0,26.0,48.0,32.0,22.0,8.0,10.0,5,6,5,4,0.00,0.0,3.000000


In [49]:
datadf.to_csv('prepeddata.csv')
Xdf = datadf.drop(['hallucinagen','depressant','stimulant'],axis = 1)

In [50]:
poly = PolynomialFeatures(degree=3,interaction_only=True)
Xd = poly.fit_transform(Xdf)
Xdf = pd.DataFrame(Xd,columns=poly.get_feature_names(Xdf.columns))

In [51]:
Xdf = Xdf.drop(['1'], axis = 1)


In [52]:

vtc = pkl.load(open('Models/vtc.pkl','rb'))
selector = RFE(LogisticRegression(max_iter=1000,solver = 'liblinear'),n_features_to_select=43,verbose = True)

In [53]:
#datadf['stimulant'] = datadf['stimulant'].apply(lambda x: 'None' if x == 0 else 'low' if x <= 3 else 'high')
selector.fit(Xdf,datadf.stimulant)

Fitting estimator with 377 features.


ValueError: Unknown label type: 'continuous'

In [ ]:
polydf = Xdf[Xdf.columns[selector.get_support()]]

In [ ]:
polydf.to_csv('polydf.csv')

In [54]:
df.describe()

,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Amphet,Amyl,...,Legalh,Lsd,Meth,Mushroom,Nicotine,Semer,VSA,stimulant,depressant,hallucinagen
count,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,...,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000
mean,35.921485,39.576127,45.762334,42.866313,41.437135,4.800531,6.560743,4.635013,1.340584,0.606897,...,1.355968,1.061538,0.826525,1.187268,3.200531,0.009549,0.433422,0.842175,1.036074,1.187622
std,9.135869,6.771769,6.579641,6.438106,6.966625,2.121445,2.696453,1.331253,1.783587,1.064210,...,1.789647,1.491103,1.646632,1.466341,2.413932,0.159331,0.962375,1.003688,1.170803,1.310756
min,12.000000,16.000000,24.000000,12.000000,17.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.000000,35.000000,41.000000,39.000000,37.000000,3.000000,5.000000,4.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,36.000000,40.000000,46.000000,43.000000,42.000000,4.000000,7.000000,5.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.500000,1.000000,0.666667
75%,42.000000,44.000000,51.000000,48.000000,46.000000,6.000000,9.000000,6.000000,2.000000,1.000000,...,3.000000,2.000000,0.000000,2.000000,6.000000,0.000000,0.000000,1.500000,1.500000,2.000000
max,60.000000,59.000000,60.000000,60.000000,59.000000,10.000000,11.000000,6.000000,6.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,4.000000,6.000000,5.250000,5.500000,5.666667
